In [1]:
from loader import tanaka_corpus
from prep import make_data_set

In [2]:
train_sentences = tanaka_corpus(is_train=True)
test_sentences = tanaka_corpus(is_train=False)

In [3]:
data_set = make_data_set(*train_sentences, *test_sentences)

In [8]:
!poetry run python -m spacy download en
!poetry run python -m spacy download de

     |████████████████████████████████| 12.1MB 997kB/s eta 0:00:01    |█████████▍                      | 3.5MB 3.0MB/s eta 0:00:03     |██████████████████████          | 8.3MB 802kB/s eta 0:00:05     |████████████████████████████▍   | 10.7MB 821kB/s eta 0:00:02     |████████████████████████████▍   | 10.7MB 821kB/s eta 0:00:02
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.0-cp37-none-any.whl size=12048610 sha256=cd4ae4a6111641fe03324aa7105c0a5d7c13da4ab3248fb5403f7f9809ff641a
  Stored in directory: /private/var/folders/_0/f9sp8z895399jqc5ryb95d3m0000gn/T/pip-ephem-wheel-cache-szeqrqve/wheels/be/ec/e4/29a68c7de525fcfa2d898cc27d2b945fe7cd966feeb2d85e0b
Successfully built en-core-web-sm
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/ryutayamamoto/.ghq/github.com/Ryuta-Yamamoto/ML-study/.venv/lib/python3.7/site-packages/

  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.3.0-cp37-none-any.whl size=14907580 sha256=012c10ed4cf7155aae0a551ea2f691d6cdc5a919def5552c3a567005e8400e96
  Stored in directory: /private/var/folders/_0/f9sp8z895399jqc5ryb95d3m0000gn/T/pip-ephem-wheel-cache-_ngm7tjk/wheels/db/f3/1e/0df0f27eee12bd1aaa94bcfef11b01eca62f90b9b9a0ce08fd
Successfully built de-core-news-sm
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/Users/ryutayamamoto/.ghq/github.com/Ryuta-Yamamoto/ML-study/.venv/lib/python3.7/site-packages/de_core_news_sm
-->
/Users/ryutayamamoto/.ghq/github.com/Ryuta-Yamamoto/ML-study/.venv/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [9]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            tokenizer_language="de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:03<00:00, 402kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 93.0kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 84.6kB/s]


Counter({'zwei': 3873,
         'junge': 2068,
         'weiße': 146,
         'männer': 1662,
         'sind': 490,
         'im': 3107,
         'freien': 475,
         'in': 11895,
         'der': 4990,
         'nähe': 383,
         'vieler': 2,
         'büsche': 5,
         '.': 28809,
         'mehrere': 580,
         'mit': 8843,
         'schutzhelmen': 33,
         'bedienen': 9,
         'ein': 18851,
         'antriebsradsystem': 1,
         'kleines': 772,
         'mädchen': 2121,
         'klettert': 171,
         'spielhaus': 2,
         'aus': 910,
         'holz': 61,
         'mann': 7805,
         'einem': 13711,
         'blauen': 992,
         'hemd': 1202,
         'steht': 1778,
         'auf': 8745,
         'einer': 6765,
         'leiter': 56,
         'und': 8925,
         'putzt': 43,
         'fenster': 148,
         'stehen': 939,
         'am': 911,
         'herd': 17,
         'bereiten': 79,
         'essen': 377,
         'zu': 1909,
         'grün':

In [10]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [15]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [23]:
import random
from typing import Tuple, Optional

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

In [ ]:
BATCH_SIZE = 128

def gen_batch()

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [92]:
# src_vcab = 1000
# trg_vcab = 1000

class MyTransformer(nn.Module):
    def __init__(
        self, 
        src_emb: nn.Module, 
        trg_emb: nn.Module,
        transformer: nn.Module,
        device: Optional[torch.device]
    ):
        super().__init__()
        self.src_emb = src_emb
        self.trg_emb = trg_emb
        self.transformer = transformer
        self.out = nn.Linear(
            in_features=transformer.d_model,
            out_features=trg_emb.num_embeddings
        )
        if not device:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        
    def forward(self, src: Tensor, trg: Tensor):
        src_tensor = self.src_emb(src)
        trg_tensor = self.trg_emb(trg)
        hidden = self.transformer(src_tensor, trg_tensor)
        return self.out(hidden)
        

def make_transformer(
        src_vcab: int,
        trg_vcab: int,
        embedding_dim: int,
        n_head: int = 16,
        num_encoder_layers: int = 3
):
    src_emb = nn.Embedding(
        num_embeddings=src_vcab, 
        embedding_dim=embedding_dim
    )
    trg_emb = nn.Embedding(
        num_embeddings=trg_vcab,
        embedding_dim=embedding_dim
    )
    transformer = nn.Transformer(
        d_model=embedding_dim,
        nhead=n_head, 
        num_encoder_layers=num_encoder_layers
    )
    return MyTransformer(
        src_emb=src_emb,
        trg_emb=trg_emb,
        transformer=transformer
    )
        

In [93]:
model = make_transformer(
    src_vcab=len(SRC.vocab.itos),
    trg_vcab=len(TRG.vocab.itos),
    embedding_dim=256,
)

In [87]:
optimizer = optim.Adam(model.parameters())

In [94]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

KeyboardInterrupt: 

In [54]:
e = next(iter(train_iterator))


In [58]:
e.src.size(-1)

128

In [52]:
next(iter(train_iterator)).trg.size()

torch.Size([26, 128])

In [26]:
import math
import time


def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [22]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

NameError: name 'nn' is not defined

In [105]:
Tensor(data_set.train_origin.seqs.T)[:3]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [113]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, src, trg):
        self.src = Tensor(src)
        self.trg = Tensor(trg)
        
    def __getitem__(self, n):
        return self.src[n], self.trg[n]
    
    def __len__(self):
        return len(self.src)

In [114]:
data = TextDataset(data_set.train_origin.seqs.T, data_set.train_trans.seqs.T)

In [110]:
from torch.utils.data import DataLoader

In [115]:
loader = DataLoader(data, batch_size=16)

In [119]:
def train(model: nn.Module,
          loader: DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(loader):
        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(loader)


In [120]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)